In [ ]:
%matplotlib inline 

In [ ]:
import warnings
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import matplotlib.animation as mpla
import requests
import json
from pprint import pprint
import geopandas as gpd

warnings.filterwarnings("ignore")

# Find out charge status percentages by LGA

In [ ]:
xlsx = pd.ExcelFile("../Data/LGA_Criminal_Incidents_Year_Ending_September_2021.xlsx")
charge_status_df = pd.read_excel(xlsx, "Table 05")
charge_status_df

In [ ]:
# combining charges laid and no charges laid to Solved status
charge_status_df["Charge Status"] = charge_status_df["Charge Status"].str.replace("Charges laid" , "Solved")
charge_status_df["Charge Status"] = charge_status_df["Charge Status"].str.replace("No charges laid" , "Solved")
charge_status_df

In [ ]:
# Add up new total columns
charge_by_types = charge_status_df.groupby(["Local Government Area", "Year","Charge Status"])
total_incidents = charge_status_df.groupby(["Local Government Area", "Year"])
charge_by_types_df = pd.DataFrame({"Incidents Combined" : charge_by_types["Incidents Recorded"].sum().astype(int)})

charge_by_types_df["Total Incidents-LGA"] = total_incidents["Incidents Recorded"].sum().astype(int)
charge_by_types_df.reset_index(level = [0,1,2], inplace= True)                                
charge_by_types_df

In [ ]:
# Calculate percentage charge status:
charge_by_types_df["Status Percentage"] = round(charge_by_types_df["Incidents Combined"] 
                                                / charge_by_types_df["Total Incidents-LGA"] * 100, 2)
charge_by_types_df

# Combine police stations number and charge status

In [ ]:
population_df =  pd.read_csv("../output_data/Population_by_LGA_2012-2021.csv")

In [ ]:
population_df

In [ ]:
LGA_charge_status_df = pd.merge(population_df, charge_by_types_df , how = "right", on = ["Year", "Local Government Area"])
LGA_charge_status_df

In [ ]:
LGA_charge_status_2021_df = LGA_charge_status_df.loc[LGA_charge_status_df["Year"] == 2021]
LGA_charge_status_2021_df

In [ ]:
police_stations_df = pd.read_csv("../output_data/Number_of_police_stations_in_each_LGA_2021.csv")

In [ ]:
LGA_charge_status_ps_df = pd.merge(police_stations_df, LGA_charge_status_2021_df , how = "right", on = "Local Government Area")

LGA_charge_status_ps_df = LGA_charge_status_ps_df.rename(columns = {"Incidents Combined" : "Incidents Recorded"} )
LGA_charge_status_ps_df

In [ ]:
charge_solved_df = LGA_charge_status_ps_df.loc[LGA_charge_status_ps_df["Charge Status"] == "Unsolved"]
unsolved_percentage = charge_solved_df["Status Percentage"]
mean_charge = round(np.mean(unsolved_percentage),1)
std_charge = round(np.std(unsolved_percentage, ddof = 0),1)
print(f"Mean solved: {mean_charge}%")
print(f"Median solved: {round(np.median(unsolved_percentage),1)}%")
print(f"Standard deviation: {std_charge}%")

print(f"68% of LGA has unsolved charge percentage between {mean_charge - std_charge}% and {mean_charge + std_charge}%")
print(f"95% of LGA has unsolved charge percentage between {mean_charge - 2 * std_charge}% and {mean_charge + 2 * std_charge}%")
print(f"99.7% of LGA has unsolved charge percentage between {round(mean_charge - 3 * std_charge,1)}% and {mean_charge + 3 * std_charge}%")

In [ ]:
charge_solved_df.sort_values("Status Percentage",inplace=True)
charge_solved_df = charge_solved_df.reset_index()
charge_solved_df = charge_solved_df.drop(columns=["index","Police Stations in LGA","Year","Total Population","Charge Status"])
charge_solved_df

In [ ]:
plt.hist(unsolved_percentage)
plt.xlabel("Unsolved charges - percentage")
plt.ylabel("Counts")
plt.style.use("default")
plt.title("LGA Charge status")
plt.show()
print(st.normaltest(unsolved_percentage.sample(50)))

In [ ]:
fig , ax = plt.subplots()
ax.set_title("Unsolved charges percentage - all LGA in 2021")
ax.set_ylabel("Unsolved charges percentage")
ax.boxplot(unsolved_percentage)
plt.show()

### Conclusion:
Data is normally distributed, no outliner identified

In [ ]:
# setting up function to show scatter plot by year
def scatter_plot(df, year, chargetype, col1, col2):
    new_df = df.loc[(df["Charge Status"] == chargetype) & (df["Year"] == year)]
    title = str(f"{col1} vs {col2} \n {chargetype} charges - {year}")
    new_df.plot(col1, col2, kind = "scatter", xlabel = col1, title = title)
                
    (slope, intercept, rvalue, pvalue, stderr) = linregress(new_df[col1],new_df[col2])
    equation = f"y = {round(slope,2)}x + {round(intercept,2)}"
    x = np.arange(0,new_df[col1].max()+1,1)
    y = [slope * x1 + intercept for x1 in x]
    
    plt.plot(x, y, color = "red" )
    plt.tight_layout()
    plt.style.use("Solarize_Light2")
    plt.annotate(equation, (new_df[col1].max()/3*2 , new_df[col2].max()/3), color = "red")
    print(f"r-value is %.2f" %rvalue)

In [ ]:
scatter_plot(LGA_charge_status_ps_df, 2021, "Unsolved", "Police Stations in LGA","Status Percentage")


In [ ]:
scatter_plot(LGA_charge_status_ps_df, 2021, "Solved", "Police Stations in LGA","Status Percentage")

In [ ]:
scatter_plot(LGA_charge_status_ps_df, 2021, "Unsolved", "Total Population", "Status Percentage")

In [ ]:
scatter_plot(LGA_charge_status_ps_df, 2021, "Solved", "Total Population", "Status Percentage")

# Getting charge percentage and population dataframe for regions

In [ ]:
# Allocate LGA data to regions
LGA_by_Regions_df = pd.read_csv("../Output_data/LGA_by_Regions_2012-2021.csv")
LGA_by_Regions_df

In [ ]:

LGA_charge_status_df = LGA_charge_status_df.rename(columns = {"Total Population" : "Total Population-LGA",
                                                              "Incidents Combined" :"Incidents Combined-LGA",
                                                              "Status Percentage" : "Status Percentage-LGA"})

In [ ]:

region_charge_status_df = pd.merge(LGA_by_Regions_df, LGA_charge_status_df, how = "right", on = ("Year","Local Government Area"))
region_charge_status_df
                                   

In [ ]:
Regions_population_df = pd.read_csv("../Output_data/Population_by_Regions_2012-2021.csv")
region_charge_pop_df = pd.merge(region_charge_status_df, Regions_population_df , how = "left" , on = ("Police Region", "Year"))
region_charge_pop_df = region_charge_pop_df.rename(columns = {"Total Population":"Total Population-Regions"})

regions_charge_gb = region_charge_pop_df.groupby(["Year", "Police Region", "Charge Status"])

incident_by_charge = regions_charge_gb["Incidents Combined-LGA"].sum()
total_incident_by_region = regions_charge_gb["Total Incidents-LGA"].sum()

regions_charge_df = pd.DataFrame({"Status Percentage-Region": incident_by_charge/total_incident_by_region*100})
regions_charge_df.reset_index(level = [0,1,2],inplace = True)                 
region_charge_pop_df = pd.merge(regions_charge_df, region_charge_pop_df, how = "right" , on = ("Police Region", "Year", "Charge Status"))
region_charge_pop_df

In [ ]:
unsolved_region_charge_pop_df = region_charge_pop_df.loc[region_charge_pop_df["Charge Status"] == "Unsolved"]
unsolved_region_charge_pop_df = unsolved_region_charge_pop_df.drop_duplicates(subset = ("Year","Police Region"), keep = "first")

unsolved_region_charge_pop_df = unsolved_region_charge_pop_df[["Year","Police Region","Status Percentage-Region","Total Population-Regions"]]
unsolved_region_charge_pop_df.head()

In [ ]:
# fig, ax = plt.subplot(figsize = (15,15))

for name, data in unsolved_region_charge_pop_df.groupby("Police Region"):
    plt.plot(data["Year"], data["Status Percentage-Region"], label = name)
plt.xlabel("Year")
plt.ylabel("Status Percentage (%)")
plt.ylim(0,100)
plt.title("Unsolved charges by regions")
plt.style.use("seaborn-notebook")
plt.legend()
plt.show()

In [ ]:
# fig, ax = plt.subplot(figsize = (15,15))

for name, data in unsolved_region_charge_pop_df.groupby("Police Region"):
    plt.plot(data["Year"], data["Total Population-Regions"], label = name)
plt.xlabel("Year")
plt.ylabel("Population")
plt.title("Total Population growth")
plt.style.use("seaborn-notebook")
plt.legend()
plt.show()

# Draw LGA boundaries and show charges status on the map

In [ ]:
# # getting LGA boundaries
# url = "https://data.gov.au/geoserver/vic-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_bdf92691_c6fe_42b9_a0e2_a4cd716fa811&outputFormat=json"
# response = requests.get(url).json()

# # # Save response to local Data folder
# with open('../Data/LGA_boundaries.json', 'w') as json_file:
#     json.dump(response, json_file)

In [ ]:
boundaries = gpd.read_file('../Data/LGA_boundaries.json')

LGA_boundaries = boundaries[["vic_lga__3", "geometry"]]
LGA_boundaries = LGA_boundaries.rename(columns = {"vic_lga__3": "LGA"} )

LGA_boundaries["LGA"] = LGA_boundaries["LGA"].str.title()

LGA_boundaries["LGA"] = LGA_boundaries["LGA"].replace("Colac Otway","Colac-Otway")

LGA_boundaries

In [ ]:
solved_charge_percentage = LGA_charge_status_ps_df.loc[LGA_charge_status_ps_df["Charge Status"] == "Unsolved"]
solved_charge_percentage = solved_charge_percentage[["Local Government Area", "Status Percentage"]]
solved_charge_percentage = solved_charge_percentage.rename(columns = {"Local Government Area":"LGA"})
solved_charge_percentage

In [ ]:
LGA_boundaries = LGA_boundaries.merge(solved_charge_percentage,how = "left" , on = "LGA")

In [ ]:
LGA_boundaries.loc[LGA_boundaries["Status Percentage"].isna()]

In [ ]:
LGA_boundaries = LGA_boundaries.dropna()

In [ ]:
LGA_boundaries

In [ ]:
len(LGA_boundaries)

In [ ]:
# There should be 79 LGAs
LGA_boundaries.drop_duplicates(subset="LGA", keep = "first", inplace=True)

In [ ]:
with open('../Data/VMFEAT_POLICE_STATION.json') as f:
    js = json.load(f)
policeStations = gpd.read_file('../Data/VMFEAT_POLICE_STATION.json')

In [ ]:
f, ax = plt.subplots(1, figsize = (15,15))
# plt.style.use("seaborn-notebook") #"Solarize_Light2"
LGA_boundaries.plot("Status Percentage", cmap = "seismic", ax = ax, legend = True, 
                        edgecolor = "black", alpha = 0.5, legend_kwds={'label': "Charge percentage(%)",
                                                                  'orientation': "horizontal"
                                                                      })
policeStations.plot(ax = ax, color = "blue", marker = "*", markersize = 10)
plt.title("Police stations in Victoria", fontdict = {'fontsize' : 16, 'color': "blue"})
plt.style.use("seaborn-notebook")#"Solarize_Light2"
# plt.show_legend()
plt.show()

In [ ]:
# # Map for readme file
# LGA_boundaries.explore(scheme="naturalbreaks")

# Getting Administrative Region boundaries

In [ ]:
# # getting regions boundaries
# url = "https://vicroadsopendata-vicroadsmaps.opendata.arcgis.com/datasets/a6e912737f234c5b825242ceb19a33b1_0.geojson?outSR=%7B%22latestWkid%22%3A3111%2C%22wkid%22%3A102171%7D"
# response = requests.get(url).json()

# # # Save response to local Data folder
# with open('../Data/region_boundaries.json', 'w') as json_file:
#     json.dump(response, json_file)

In [ ]:
region_boundaries = gpd.read_file('../Data/region_boundaries.json')

In [ ]:
region_boundaries

In [ ]:
region_boundaries.plot(column = "REG_NAME",cmap = "Paired",legend = True, edgecolor = "red", alpha = 0.5)
plt.title("Administrative boundaries in Victoria", fontdict = {'fontsize' : 16, 'color': "blue"})

# Getting Administrative Region boundaries

In [ ]:
# getting region boundaries
LGA_by_Regions_df = LGA_by_Regions_df.loc[LGA_by_Regions_df["Year"]==2021]
LGA_by_Regions_df = LGA_by_Regions_df.rename(columns = {"Local Government Area":"LGA"})

In [ ]:
LGA_by_Regions_df

In [ ]:
regions = LGA_boundaries.merge(LGA_by_Regions_df, how = "left", on = "LGA")

In [ ]:
regions

In [ ]:
regions = regions[["Police Region" , "geometry"]]
PSA_regions_boundaries = regions.dissolve(by = "Police Region")

PSA_regions_boundaries.reset_index(level = [0], inplace= True)

PSA_regions_boundaries

In [ ]:
PSA_regions_boundaries.plot(column = "Police Region",cmap = "Paired",legend = True, edgecolor = "red", alpha = 0.5)
plt.title("Police Service Area boundaries in Victoria", fontdict = {'fontsize' : 16, 'color': "blue"})

In [ ]:
f, ax = plt.subplots(1, figsize = (15,15))
plt.style.use("seaborn-notebook") #"Solarize_Light2"
LGA_boundaries.plot("Status Percentage", cmap = "seismic", ax = ax, legend = True, 
                        edgecolor = "black", alpha = 0.5, legend_kwds={'label': "Charge percentage(%)",
                                                                  'orientation': "horizontal",
                                                                      })
PSA_regions_boundaries.plot("Police Region", ax = ax, facecolor = "none", edgecolor = "black", linewidth = 1.2, legend = True)
policeStations.plot(ax = ax, color = "blue", marker = "*", markersize = 10)
plt.title("Police stations in Victoria", fontdict = {'fontsize' : 16, 'color': "blue"})
